<a href="https://colab.research.google.com/github/Joongeun/Internship/blob/main/Fixed_Bag_Of_Words_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_text(sentences):
  cleaned_sentence = ""
  sentences = sentences.lower().split()
  ##removing stop words
  words = [i for i in sentences if i not in stopwords.words('english')]
  words = " ".join(words)               ##joining our words back to sentences
  return words

In [ ]:
def preprocess(length, ngram):
  X = df['selftext'][0:length].tolist()
  Y = df['labels'][0:length].tolist()
  for i, post in enumerate(X):
    X[i] = clean_text(post)
  if ngram == "binary":
    vectorizer = CountVectorizer(binary=True)
  elif ngram == "unigram":
    vectorizer = CountVectorizer()
  else:
    vectorizer = CountVectorizer(ngram_range = (2, 2), binary=True) #Bigram/binary
  bag = vectorizer.fit_transform(X)
  #len(bag) == 100 - vectors of posts
  bag = bag.toarray()
  skf = StratifiedKFold(n_splits=10)
  skf.get_n_splits(bag, Y)
  X_train, y_train, X_test, y_test = [], [], [], []
  for i, (train_index, test_index) in enumerate(skf.split(bag, Y)):
    xtrain, ytrain, xtest, ytest = [], [], [], []
    for i in train_index:
      xtrain.append(bag[i])
      ytrain.append(Y[i])
    for i in test_index:
      xtest.append(bag[i])
      ytest.append(Y[i])
    X_train.append(xtrain)
    y_train.append(ytrain)
    X_test.append(xtest)
    y_test.append(ytest)
  return X_train, y_train, X_test, y_test
  #len(X_train, y_train) == 10, each of the 10 sublists have 90 np vector arrays (posts)
  #len(X_test, y_test) == 10, 10 np arrays per fold - 100 posts total
# print(X_test)
# print(y_test)
# print(len(X_test))
# print(X_test[0])
# print(len(X_test[0]))

In [ ]:
def classify(X_train, y_train, X_test, y_test, length, count, label):
  w = {0:length-count, 1:count}
  lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter = 1000000, class_weight = w)

  # Use metrics.accuracy_score to measure the score
  #'macro' favors minority, 'weighted' favors majority, 'micro' favors none (when you have multiclass)
  scores = []
  for i in range(0, 10):
    #Get 9 folds for training
    xtrain = X_train[i]
    ytrain = y_train[i]
    lr.fit(xtrain, ytrain)

    xtest = X_test[i]
    ytest = y_test[i]
    y_predict = lr.predict(xtest)
    if label in [1.0, 4.0]:
      score = f1_score(ytest, y_predict, average='micro', labels=np.unique(y_predict))
    else:
      score = f1_score(ytest, y_predict, average='macro', labels=np.unique(y_predict))
    scores.append("%.3f" %score)
  return scores

In [ ]:
df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')
# for i in list(set(df.labels))[:6]:
for i in [2.0, 3.0, 4.0]:
  count = 0
  for ind in df.index:
    # float 0.0, 1.0, 2.0, 3.0, 4.0, 5.0 - represents: general, question, advice, encouragement, experience, bragging posts
    if df.loc[ind, "labels"] != i:
      df.loc[ind, "labels"] = 0
    else:
      df.loc[ind, "labels"] = 1
      count += 1
  length = 3000
  unigram, bigram, binary = [], [], []
  while length < 3100:
    X_train, y_train, X_test, y_test = preprocess(length, "unigram")
    unigram = classify(X_train, y_train, X_test, y_test, length, count, i)
    X_train, y_train, X_test, y_test = preprocess(length, "bigram")
    bigram = classify(X_train, y_train, X_test, y_test, length, count, i)
    X_train, y_train, X_test, y_test = preprocess(length, "binary")
    binary = classify(X_train, y_train, X_test, y_test, length, count, i)
    length += 100
  print("Scores for", i, ": \n")
  sum = 0
  for i in unigram:
      print(i)
      sum += float(i)
  print("Average: " + str(sum))
  print("-----------------------")
  sum = 0
  for i in bigram:
    print(i)
    sum += float(i)
  print("Average: " + str(sum/10))
  print("-----------------------")
  sum = 0
  for i in binary:
    print(i)
    sum += float(i)
  print("Average: " + str(sum/10))
  print("\n")
  df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')

Scores for 2.0 : 

0.801
0.586
0.751
0.770
0.700
0.661
0.719
0.670
0.706
0.783
Average: 7.147000000000001
-----------------------
0.973
0.973
0.973
0.546
0.546
0.546
0.973
0.973
0.546
0.973
Average: 0.8022
-----------------------
0.740
0.630
0.764
0.748
0.678
0.630
0.679
0.670
0.653
0.706
Average: 0.6898


Scores for 3.0 : 

0.488
0.606
0.534
0.484
0.630
0.656
0.534
0.520
0.638
0.534
Average: 5.624
-----------------------
0.978
0.978
0.978
0.978
0.978
0.978
0.976
0.976
0.976
0.976
Average: 0.9771999999999998
-----------------------
0.556
0.606
0.483
0.486
0.630
0.561
0.586
0.581
0.647
0.484
Average: 0.562


Scores for 4.0 : 

0.727
0.740
0.687
0.740
0.700
0.737
0.673
0.693
0.717
0.710
Average: 7.124
-----------------------
0.693
0.703
0.657
0.673
0.677
0.687
0.693
0.670
0.670
0.677
Average: 0.6799999999999999
-----------------------
0.697
0.747
0.707
0.737
0.730
0.740
0.673
0.683
0.733
0.730
Average: 0.7177




In [ ]:
#test to see how many of each post there are
import pandas as pd
df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')
labels = list(df.labels)
questions = labels.count(1.0)
advice = labels.count(2.0)
encouragement  = labels.count(3.0)
experience = labels.count(4.0)
bragging = labels.count(5.0)
print(questions, advice, encouragement, experience, bragging)
print(labels.count(0.0))
ratio = {0:2000-experience, 1:experience}
print(ratio)

1046 103 90 675 25
61
{0: 1325, 1: 675}


Test model on rest of CSV data
Error: X has 12867 features, but LogisticRegression is expecting 11214 features as input.

In [ ]:
def test(length, ngram, count, label):
  X = df['selftext'][0:length].tolist()
  Y = df['labels'][0:length].tolist()
  for i, post in enumerate(X):
    X[i] = clean_text(post)
  if ngram == "binary":
    vectorizer = CountVectorizer(binary=True)
  elif ngram == "unigram":
    vectorizer = CountVectorizer()
  else:
    vectorizer = CountVectorizer(ngram_range = (2, 2))
  bag = vectorizer.fit_transform(X)
  bag = bag.toarray()

  skf = StratifiedKFold(n_splits=10)
  skf.get_n_splits(bag, Y)

  for i, (train_index, test_index) in enumerate(skf.split(bag, Y)):
    # print(i)
    X_train, y_train, X_test, y_test = [], [], [], []
    for i in train_index:
      X_train.append(bag[i])
      y_train.append(Y[i])
    for i in test_index:
      X_test.append(bag[i])
      y_test.append(Y[i])
    (X_train, y_train, X_test, y_test) = map(np.array, (X_train, y_train, X_test, y_test))
  w = {0:2000-count, 1:count}
  lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter = 100000, class_weight = w)
  #
  # Fit the model
  #
  lr.fit(X_train, y_train)
  #
  # Create the predictions
  #
  y_predict = lr.predict(X_test)

  # Use metrics.accuracy_score to measure the score
  #'macro' favors minority, 'weighted' favors majority, 'micro' favors none (when you have multiclass)
  if label in [1.0, 4.0]:
    score = f1_score(y_test, y_predict, average='micro', labels=np.unique(y_predict))
  else:
    score = f1_score(y_test, y_predict, average='macro', labels=np.unique(y_predict))
  return "%.3f" %score, lr

In [ ]:
def a(ngram):
  vectorizer = 0
  if ngram == "binary":
    vectorizer = CountVectorizer(binary=True)
  elif ngram == "unigram":
    vectorizer = CountVectorizer()
  else: #bigram
    vectorizer = CountVectorizer(ngram_range = (2, 2))
  return vectorizer

def b(vectorizer):
  X = np.array(df['selftext'][3100:7751].tolist())
  for i, post in enumerate(X):
    X[i] = clean_text(post)

  bag = vectorizer.fit_transform(X)
  bag = bag.toarray()
  return bag

df = pd.read_csv('/content/Labeled Posts - preprocessed_csv.csv')
count = 0
i = 2.0
for ind in df.index:
  if df.loc[ind, "labels"] != i:
      df.loc[ind, "labels"] = 0
  else:
    df.loc[ind, "labels"] = 1
    count += 1

vectorizer = a("unigram")
unigram, lr1 = test(3000, "unigram", count, i)
# bigram, lr2  = test(3000, "bigram", count, i)
# vectorizer = a("binary")
# binary, lr3 = test(3000, "binary", count, i)

bag = b(vectorizer)
data = np.array(bag)

print(lr1.predict(data))
# print(lr2.predict(data))
# print(lr3.predict(data))